In [5]:
!pip install -U -q google-generativeai nbformat

In [ ]:
import io
import time
import json
import os
import nbformat
import google.generativeai as genai
from google.colab import auth, userdata
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.api_core import exceptions

# 1. AUTHENTICATION & SETUP
auth.authenticate_user()
drive_service = build('drive', 'v3')

try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=API_KEY)
    # Using the model name you specified
    model = genai.GenerativeModel('gemini-2.5-flash')
    print("✅ System Ready. Model: gemini-2.5-flash")
except Exception as e:
    print(f"❌ Setup Error: {e}")

FOLDER_ID = '1VvKZwxUfeNQCtH5VkOuFmDWCCynnzqXo'
PROGRESS_FILE = 'audit_debug_results.json'

if os.path.exists(PROGRESS_FILE):
    with open(PROGRESS_FILE, 'r') as f: audit_results = json.load(f)
else: audit_results = {}

def get_minimal_context(nb):
    """Extracts very tiny context to stay under Token limits."""
    context = ""
    # Get just the first markdown goal
    goals = [c.source for c in nb.cells if c.cell_type == 'markdown' and 'goal' in c.source.lower()]
    if goals: context += f"Goal: {goals[0][:300]}\n"
    # Get only the first 2 code cells
    codes = [c.source for c in nb.cells if c.cell_type == 'code'][:2]
    for i, c in enumerate(codes):
        context += f"Code {i+1}: {c[:300]}\n"
    return context

def run_audit():
    query = f"'{FOLDER_ID}' in parents and name contains '.ipynb' and trashed = false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])

    print(f"Found {len(files)} notebooks. {len(audit_results)} already finished.\n")

    for file in files:
        f_id, f_name = file['id'], file['name']
        if f_id in audit_results: continue

        print(f"--- [Analyzing: {f_name}] ---")

        try:
            # IMPROVED DOWNLOAD LOGIC
            # Download as bytes first
            content_bytes = drive_service.files().get_media(fileId=f_id).execute()

            # Parse from the bytes string
            nb_content = content_bytes.decode('utf-8', errors='ignore')
            nb = nbformat.reads(nb_content, as_version=4)

            context = get_minimal_context(nb)
            prompt = f"Rate notebook '{f_name}' for GitHub (1-10). Explain why in 1 sentence. Context:\n{context}"

            # --- PERSISTENT RETRY LOOP ---
            while True:
                try:
                    response = model.generate_content(prompt)
                    if response.parts:
                        res_text = response.text
                        print(f"✅ SUCCESS: {res_text}")

                        audit_results[f_id] = {"name": f_name, "rating": res_text}
                        with open(PROGRESS_FILE, 'w') as f:
                            json.dump(audit_results, f, indent=4)

                        # Conservative wait to avoid the 429 quota error
                        time.sleep(35)
                        break
                    else:
                        print("⚠️ Safety Filter Blocked this file.")
                        break

                except exceptions.ResourceExhausted:
                    print("⚠️ Quota hit! Waiting 65 seconds for full reset...")
                    time.sleep(65)
                except Exception as e:
                    print(f"❌ LLM API Error: {e}. Waiting 10s...")
                    time.sleep(10)

        except Exception as e:
            # THIS WILL NOW PRINT THE ACTUAL ERROR (e.g. Permission Denied)
            print(f"❌ READ ERROR on {f_name}: {e}")

    print("\n✅ AUDIT COMPLETE!")

if __name__ == "__main__":
    run_audit()

✅ System Ready. Model: gemini-2.5-flash
Found 44 notebooks. 0 already finished.

--- [Analyzing: Resume.ipynb] ---


❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 49.335728422s.. Waiting 10s...


❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 38.69292134s.. Waiting 10s...


❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 27.975689077s.. Waiting 10s...


❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 17.110780242s.. Waiting 10s...


...

❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 53.855343985s.. Waiting 10s...


❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 43.126720639s.. Waiting 10s...


❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 32.482032298s.. Waiting 10s...


❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 21.800497852s.. Waiting 10s...


❌ LLM API Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 11.059048019s.. Waiting 10s...
❌ LLM API Error: HTTPConnectionPool(host='localhost', port=32997): Read timed out. (read timeout=600.0). Waiting 10s...
❌ LLM API Error: HTTPConnectionPool(host='localhost', port=32997): Read timed out. (read timeout=600.0). Waiting 10s...
❌ LLM API Error: HTTPConnectionPool(host='localhost', port=32997): Read timed out. (read timeout=600.0). Waiting 10s...
❌ LLM API Error: HTTPConnectionPool(host='localhost', por